# Model Context Protocol(MCP)

The **Model Context Protocol (MCP)** is an **open standard protocol initiated by Anthropic**. Its core purpose is to **standardize how AI models, applications, and agents connect to and interact with external tools, data sources, and services**. This allows AI applications to **access real-time data, use tools, fetch data, and utilize prompts** from compliant external systems.

Think of MCP as a "**USB-C port for AI**",
providing a **standardized interface** to solve the M×N integration problem by enabling any compliant AI client to connect to any compliant MCP server. It follows a **client-server architecture** (Hosts, Clients, Servers) and is built on **JSON-RPC 2.0**, supporting transports like Stdio and HTTP with SSE.

MCP servers expose capabilities including **Tools, Resources, and Prompts**. It **standardizes the execution** of instructions generated by methods like Function Calling.

<br/>

## Why MCP is needed?

The need for the Model Context Protocol (MCP) arose primarily from the **limitations of LLMs being isolated** from real-time data, user-specific context, and external systems. Before MCP, connecting AI applications to the vast landscape of tools, databases, and services required **custom, fragmented integrations for each pair** of application and tool. This resulted in the "M×N integration problem," which was **time-consuming, difficult to scale, and led to limited functionality**.

MCP was needed to provide a **standardized, universal interface**—often likened to a "**USB-C port for AI**"—to connect AI models to *any* compliant external system. It also addresses the need to **standardize the execution phase** of LLM-generated instructions (like those from Function Calling), ensuring consistency and scalability across diverse tools. This approach **simplifies development, promotes interoperability**, and allows AI applications to access the real-world context and capabilities necessary to be truly helpful.
<hr/>

![MCP](https://norahsakal.com/assets/images/mcp_overview-641a298352ff835488af36be3d8eee52.png)
Source: [Norah Sakal](https://norahsakal.com/blog/mcp-vs-api-model-context-protocol-explained/)

<hr/>


### Core Architecture

The core architecture of MCP follows a client-server model. The key components are:
<br/><br/>
• MCP Hosts: These are AI applications or environments, such as LLM Apps or IDEs, that initiate connections and operate the MCP client.
<br/><br/>
• MCP Clients: Located within the host application, clients maintain a one-to-one connection with servers. They act as intermediaries, facilitating communication between the MCP host and servers, sending requests, seeking information about server services, and handling LLM routing and orchestration.
<br/><br/>
• MCP Servers: These are lightweight programs that act as a gateway, allowing the MCP client to interact with external services and execute tasks. Servers connect to data sources and tools (like Google Drive, Slack, or databases) and expose specific capabilities

<hr/>

![Corearchitecture.png](https://composio.dev/wp-content/uploads/2025/03/mcp-matt.jpeg)
Source: [Composio.dev Blogs](https://composio.dev/blog/what-is-model-context-protocol-mcp-explained/)

<hr/>

## Function Calling vs MCP

Function Calling and the Model Context Protocol (MCP) are distinct but **complementary** frameworks for integrating Large Language Models (LLMs) with external systems. They represent **two phases** in the process of enabling LLMs to interact with the real world.

**Key Differences Summarized:**


| Feature           | Function Calling                                 | MCP (Model Context Protocol)                                   |
| :---------------- | :----------------------------------------------- | :------------------------------------------------------------- |
| **Purpose**       | Translates prompts into structured instructions. | Standardizes execution and response handling of instructions. |     |
| **Focus**         | Generating action-ready commands from natural language. | Managing tool discovery, invocation, and response handling. |
| **Control**       | LLM provider (e.g., Google Gemini, OpenAI, Anthropic).     | External system/application handling LLM integration.    |             |
| **Output/Protocol** | Varies by LLM vendor (JSON-based). | Uses a standardized protocol (JSON-RPC 2.0). |
| **Role in Workflow**| "Ordering the task" (Translation phase). | "Executing the task" (Execution phase).    |
| **Architecture**  | Part of the LLM's output structure. | Client-server architecture connecting hosts, clients, and servers. |
| **Statefulness**  | Typically a stateless request/response outcome from the LLM | Can support stateful connections and interactive workflows |
| **Standardization** | No universal standard (frameworks like LangChain help manage variations). | Provides a consistent execution framework and ensures interoperability across tools. |
| **Complexity/Setup** | Simpler, more direct method. | Requires setup of MCP clients and servers, potentially higher initial complexity. |       |
| **Security**      | Relies on external API security and execution control management. | Designed with inherent security measures (host-mediated, access controls, user approval). |

**Working Together:**

Function Calling and MCP work together seamlessly. The application acts as an intermediary, translating the LLM's function call output (which varies by vendor) into a standardized MCP request (using JSON-RPC) that the MCP server can understand and execute. MCP then handles the execution of the tool and returns the result in a structured format, which the application can feed back to the LLM or the user.

Think of it like a universal adapter for AI applications, similar to USB-C for physical devices. While Function Calling allows LLMs to translate prompts into *what* needs to be done (the command), MCP standardizes *how* it gets done (the execution across different tools and data sources). This combined approach allows for more efficient, scalable, and flexible AI-powered systems.

# Get Started

## Setup
This lab needs a special kernel to run, please run the following cell.
**NOTE: You can skip this step if you have already built the ADK Kernel from the previous Lab**

In [ ]:
!echo "Kernel installation started."
!cd ../../../.. && make adk_mcp_a2a_kernel > /dev/null 2>&1
!echo "Kernel installation completed."

### Set Google Cloud project information and initialize Vertex AI SDK

To get started using Vertex AI, you must have an existing Google Cloud project and [enable the Vertex AI API](https://console.cloud.google.com/flows/enableapi?apiid=aiplatform.googleapis.com).

Learn more about [setting up a project and a development environment](https://cloud.google.com/vertex-ai/docs/start/cloud-environment).

In [ ]:
GCP_PROJECTS = !gcloud config get-value project
PROJECT_ID = GCP_PROJECTS[0]
LOCATION = "us-central1"

import os
os.environ["REGION"]=REGION
os.environ["PROJECT_ID"]=PROJECT_ID
os.environ.setdefault("GOOGLE_CLOUD_PROJECT", PROJECT_ID)
os.environ.setdefault("GOOGLE_CLOUD_LOCATION", LOCATION)
os.environ.setdefault("GOOGLE_GENAI_USE_VERTEXAI", "True")

# Demo

* This Demo uses [FastMCP](https://gofastmcp.com/getting-started/welcome) to create an Inventory Management tool MCP Server.

* An ADK based agent act as MCP host and uses a MCP client([ADK MCP Toolset](https://google.github.io/adk-docs/tools/mcp-tools/#step-1-attach-the-fastmcp-server-to-your-adk-agent-via-mcptoolset:~:text=to%20your%20ADK%20agent%20via%20MCPToolset-,%C2%B6,-Create%20agent.py%20in%20./adk_agent_samples/fastmcp_agent)) to communicate with the MCP Server and fetches the available tools.

* The ADK agent uses the tools from MCP server to answer inventory related request from users.

### Create Directory for the project

In [ ]:
!mkdir mcp-demo

# Download dummy Inventory data

In [ ]:
!wget https://raw.githubusercontent.com/solidate/mcp-demo/refs/heads/main/sku_data.csv -O mcp-demo/sku_data.csv

# MCP Server code usign FastMCP

In [ ]:
%%writefile mcp-demo/operations.py

import csv
import os
import asyncio
from fastmcp import FastMCP
from pydantic import  Field
from pydantic.types import Literal

# Define the path to the CSV file relative to this script
# Assuming operations.py and sku_data.csv are in the same directory 'agents/inventory/'
CSV_FILE_PATH = os.path.join(os.path.dirname(__file__), 'sku_data.csv')


global mcp
mcp = FastMCP(
    name="Inventory MCP Server",
    instructions="""
    This Server provides inventory related data and helps in
    updating the quantity of a specific SKU.

    Call list_skus() to get the list of all the SKUs or any details related the items/SKUs.
    Call update_sku_qty(sku_id) to update the quantity of a specific SKU.
    """
)


@mcp.tool()
def list_skus(sku_name: str = Field("*", description="Name of the SKU")):
    """
    Reads the SKU data from the CSV file and returns it as a list of dictionaries.
    If you are asked for the available items or any enquiry about the items/SKUs, call this function and
    return only the consumer freindly information like id, name and its cost and available quantity.
    """
    if not os.path.exists(CSV_FILE_PATH):
        return {"error": "SKU data file not found."}

    skus = []
    try:
        with open(CSV_FILE_PATH, mode='r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            for row in reader:
                skus.append(row)
        return {"skus": skus}
    except Exception as e:
        return {"error": f"Failed to read SKU data: {str(e)}"}

@mcp.tool()
def update_sku_qty(
            sku_id: str = Field(..., description="The SKU ID of the product to update."),
            quantity: int = Field(..., description="The quantity to be added or removed from the SKU.", ge=0),
            sign: int = Field(1, description="1 for adding and -1 for removing.")
                ):
    """
    Updates the quantity of a specific SKU in the CSV file.
    If you are asked for placing or returning/cancelling an order, call this function.

    Args:
        sku_id (str): The SKU ID of the product to update.
        quantity (int): The new quantity for the SKU.
        sign (int): 1 for adding and -1 for removing.

    Returns:
        dict: A message indicating success or failure.
    """
    if not os.path.exists(CSV_FILE_PATH):
        return {"error": "SKU data file not found."}

    if not isinstance(quantity, int):
        return {"error": "Invalid quantity. Must be a non-negative integer."}

    rows = []
    updated = False
    fieldnames = []

    try:
        with open(CSV_FILE_PATH, mode='r', newline='', encoding='utf-8') as csvfile:
            reader = csv.DictReader(csvfile)
            fieldnames = reader.fieldnames
            if not fieldnames: # Handle empty or malformed CSV
                return {"error": "CSV file is empty or has no header."}

            quantity *= sign

            for row in reader:
                if row.get('SKU') == sku_id:
                    row['QuantityOnHand'] = int(row['QuantityOnHand']) + int(quantity)
                    # Potentially update 'Status' based on new quantity vs ReorderLevel
                    if 'ReorderLevel' in row and quantity <= int(row.get('ReorderLevel', 0)):
                        row['Status'] = 'Low Stock'
                    elif 'ReorderLevel' in row and quantity > int(row.get('ReorderLevel', 0)):
                        row['Status'] = 'In Stock'
                    updated = True
                    updated_qty = row['QuantityOnHand']
                rows.append(row)

        if not updated:
            return {"error": f"SKU ID '{sku_id}' not found."}

        with open(CSV_FILE_PATH, mode='w', newline='', encoding='utf-8') as csvfile:
            writer = csv.DictWriter(csvfile, fieldnames=fieldnames)
            writer.writeheader()
            writer.writerows(rows)

        return {"message": f"Quantity for SKU '{sku_id}' updated to {updated_qty}."}
    except Exception as e:
        return {"error": f"Failed to update SKU quantity: {str(e)}"}

if __name__ == '__main__':
    asyncio.run(mcp.run_sse_async(
                        host="0.0.0.0",
                        port=4200,
                        path="/inventory",
                        )
    )

### Setup MCP Server

#### Step 1: Open the Built-in Terminal

First, open the integrated terminal in your Vertex AI Workbench interface.
 - Go to File -> New -> Terminal.

#### Step 2: Start a New tmux Session
In the terminal you just opened, check current directory:
```bash
pwd
```
change your current driectory to 
"/asl-ml-immersion/notebooks/vertex_genai/solutions/a2a/",
```bash
cd a2a
```
#### Step 3: Start a New tmux Session

In the terminal, give your session a name. Let's call it my_session.
Type the following command and press Enter:
```bash
tmux new -s demo_mcp_session
```
#### Step 4: Send Commands from Your Notebook Cell

In [ ]:
!tmux send-keys -t demo_mcp_session 'python mcp-demo/operations.py' C-m

# Create ADK based agent with MCP client
* Here MCPToolset is used to communicate with SSE server url where MCP Server runs

In [ ]:
import argparse
import asyncio
from contextlib import AsyncExitStack
from uuid import uuid4

import click
from google.adk.agents import LlmAgent
from google.adk.runners import Runner
from google.adk.sessions import InMemorySessionService
from google.adk.tools.mcp_tool.mcp_toolset import McpToolset
from google.adk.tools.mcp_tool.mcp_session_manager import StdioConnectionParams
from google.adk.tools.mcp_tool.mcp_session_manager import SseServerParams
from google.genai import types
from pydantic import BaseModel

# TODO: IMPORTANT! Change the path below to your remote MCP Server path
MCP_SERVER_URL="http://0.0.0.0:4200/inventory"

async def create_agent():
    common_exit_stack = AsyncExitStack()

    agent = LlmAgent(
        model="gemini-2.0-flash",
        name="inventory_assistant",
        description="You are a specialized assistant for inventory management.",
        instruction=(
            "Help user get answer to their queries about inventory and update "
            "the items."
        ),
        tools=[
            McpToolset(
                connection_params=SseServerParams(
                url=MCP_SERVER_URL
                ),
            )
        ],
    )
    return agent, common_exit_stack

async def ask_agent(query):
    session_service = InMemorySessionService()
    inventory_agent, exit_stack = await create_agent()

    # Create the session here, just before it's used
    session = await session_service.create_session(
        state={}, app_name="mcp_inventory_app", user_id=str(uuid4())
    )

    runner = Runner(
        app_name="mcp_inventory_app",
        agent=inventory_agent,
        session_service=session_service,
    )
    user_content = types.Content(role="user", parts=[types.Part(text=query)])

    print("Running agent...")
    events_async = runner.run_async(
        session_id=session.id, user_id=session.user_id, new_message=user_content
    )
    async for event in events_async:
        print("-" * 10)
        print(event)
        if event.is_final_response():
            final_response = "".join(part.text for part in event.content.parts)
            break

    print("-" * 10)
    print(f"Final response:")
    print("-" * 10)
    print(final_response)
    print("-" * 10)
    print("Closing MCP server connection...")
    await exit_stack.aclose()
    print("Cleanup complete.")


async def ask_inventory_agent(query):
    loop = asyncio.get_running_loop()
    await loop.create_task(ask_agent(query))

In [ ]:
def print_adk_output(event: dict):
    # Extract the first part of the content
    part = event.get("content", {}).get("parts", [{}])[0]
    author = event.get("author", "unknown_agent")
    print(f"\n--- [Event: {author}] ---")

    # Case 1: The model is calling a function/tool
    if "function_call" in part:
        call = part["function_call"]
        func_name = call.get("name", "N/A")
        func_args = call.get("args", {})
        print("Tool Call:")
        print(f"  - Function: {func_name}")
        print(f"  - Arguments: {json.dumps(func_args, indent=2)}")

    # Case 2: The tool is returning a response
    elif "function_response" in part:
        response = part["function_response"]
        func_name = response.get("name", "N/A")
        func_response = response.get("response", {})
        print("Tool Response:")
        print(f"  - From: {func_name}")
        # Pretty print the response JSON
        print(f"  - Data: {json.dumps(func_response, indent=2)}")

    # Case 3: The model is generating a final text response
    elif "text" in part:
        text_response = part["text"].strip()
        print("Final Agent Response:")
        print(f"  -> {text_response}")

    else:
        print("Unknown step type.")

In [ ]:
await ask_inventory_agent("How many mouse are available?")

In [ ]:
await ask_inventory_agent("Order 5 wireless mouse")

In [ ]:
await ask_inventory_agent("Sell 5 units for SKU002")

In [ ]:
await ask_inventory_agent("How many mouse are available?")

In [ ]:
await ask_inventory_agent("Sell 5 units for SKU002")

In [ ]:
await ask_inventory_agent("Restock 5 units for SKU002")